#### Name : SINU S MARIAM
#### Designation : Machine Learning Intern  
#### Organization : CODSOFT
#### Batch : APRIL BATCH P43
#### # Task 3 - SPAM SMS DETECTION
#### Problem Statement:
* Build an AI model that can classify SMS messages as spam or legitimate.
* Use techniques like TF-IDF or word embeddings with classifiers like Naive Bayes, Logistic Regression, or Support Vectorm Machines to identify spam messages
* Dataset Link - https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset

In [1]:
#importing necessary libraries
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Data Path
data_file='/content/spam.csv'

In [3]:
#To find encoding pattern of the file
import chardet
with open(data_file, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'Windows-1252', 'confidence': 0.7272080023536335, 'language': ''}

In [4]:
# Importing File
data_frame=pd.read_csv(data_file,encoding='Windows-1252')

####  Performing descriptive analysis. Understand the variables and their corresponding values.

In [5]:
# Understanding the dimensions of data
data_frame.shape

(5572, 5)

In [6]:
# Understanding the Data Variables
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [7]:
# List the columns of dataset
data_frame.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [8]:
# Show the top 10 Rows of data
data_frame.head(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


##### * Column v2 contains texts of emails and v1 contains whether those emails are spam or not

In [9]:
# Checking for any Duplicated Entries
data_frame.duplicated().sum()

403

In [10]:
# Dropping the duplicate values
data_frame.drop_duplicates(keep = 'first', inplace = True)
data_frame.shape

(5169, 5)

In [11]:
#Detecting missing values
data_frame.isna().sum()

v1               0
v2               0
Unnamed: 2    5126
Unnamed: 3    5159
Unnamed: 4    5164
dtype: int64

In [12]:
#Dropping unwanted Columns from data
data_frame.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'], inplace=True )
data_frame.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [13]:
# Finding the number of spam and non-spam sms in dataset
data_frame['v1'].value_counts()

v1
ham     4516
spam     653
Name: count, dtype: int64

In [14]:
# Printing examples of ham messages
print("Some examples of ham messages:")
print(data_frame[data_frame['v1'] == 'ham']['v2'].head())

Some examples of ham messages:
0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
6    Even my brother is not like to speak with me. ...
Name: v2, dtype: object


In [15]:
# Printing examples of spam messages
print(" Some examples of spam messages:")
print(data_frame[data_frame['v1'] == 'spam']['v2'].head())

 Some examples of spam messages:
2     Free entry in 2 a wkly comp to win FA Cup fina...
5     FreeMsg Hey there darling it's been 3 week's n...
8     WINNER!! As a valued network customer you have...
9     Had your mobile 11 months or more? U R entitle...
11    SIX chances to win CASH! From 100 to 20,000 po...
Name: v2, dtype: object


In [16]:
# Labelling Spam sms as 1 and Non-Spam sms as 0
data_frame['v1'] = np.where(data_frame['v1'] == 'spam',1,0)
data_frame

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


####   Splitting data for training and testing the model

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
#First step in building the model is to identify the Feature(Input) variables and Target (Output) variable
#Features are the sms content
#Target represents the labels for spam and non-spam sms
features = data_frame['v2']
target = data_frame['v1']

In [19]:
# Splitting data for training the model and testing the model
# train size taken as 0.8
X_train, X_test, y_train, y_test = train_test_split(features, target, train_size = .8)
# Dimensions of Train and Test Data sets
print('Train set of features: ', X_train.shape)
print('Test set of features: ', X_test.shape)
print('Target for train: ', y_train.shape)
print('Target for test: ', y_test.shape)

Train set of features:  (4135,)
Test set of features:  (1034,)
Target for train:  (4135,)
Target for test:  (1034,)


### Apply Word Embedding Tecnique - CountVectorizer

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
#CountVectorizer transform text into a vector on the basis of the frequency/count of each word that occurs in entire text.
#Covert each word in Train dataset into vectors for using them in further text analysis.
cv=CountVectorizer()
X_train_vector=cv.fit_transform(X_train.values)
X_train_vector.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [22]:
#Coverting each word in Test dataset into vectors for using them in further text analysis.
X_test_vector=cv.transform(X_test.values)
X_test_vector.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [23]:
# Here Multinomial Naive Bayes model which is a supervised learning classification
#Used for the analysis of the categorical text data.
from sklearn.naive_bayes import MultinomialNB
my_model_NB=MultinomialNB()
# Fitting the model in train data set ie the MultinomialNB Model should learn from the Train Data
my_model_NB.fit(X_train_vector, y_train)

MultinomialNB()

#### Prediction using Naive Bayes model

In [24]:
# Predicting whether the mails in Test Dataset are spam or non-spam using our model from Feature Test values
y_pred_NB = my_model_NB.predict(X_test_vector)
y_pred_NB

array([0, 0, 0, ..., 0, 0, 0])

#### Testing the Naive Bayes model

In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [26]:
# find Accurancy Score = (TP+TN)/(TP+TN+FP+FN)
accuracy_score(y_test, y_pred_NB)

0.9864603481624759

In [27]:
# find Precision Score= TP/(TP+FP)
precision_score(y_test, y_pred_NB)

0.9915966386554622

In [28]:
# Recall = TP/(TP+FN)
# Recall gives the percentage of positives well predicted by our model.
recall_score(y_test, y_pred_NB)

0.9007633587786259

In [29]:
# f1_score= (Precition * Recall )/ (Precition + Recall)
f1_score(y_test, y_pred_NB)

0.944

### Support Vector Machine Classifier Model

In [30]:
from sklearn.svm import SVC
my_model_SVC= SVC()
# Fitting the model in train data set ie SVC Model should learn from the on Train Data
my_model_SVC.fit(X_train_vector, y_train)

SVC()

#### Predicting using SVC

In [31]:
# Predicting whether the sms in Test Dataset are spam or non-spam using our model from Feature Test values
y_pred_SVC = my_model_SVC.predict(X_test_vector)
y_pred_SVC

array([0, 0, 0, ..., 0, 0, 0])

#### Testing the SVC Model

In [32]:
# find Accurancy Score = (TP+TN)/(TP+TN+FP+FN)
accuracy_score(y_test, y_pred_SVC)

0.9748549323017408

In [33]:
# find Precision Score= TP/(TP+FP)
precision_score(y_test, y_pred_SVC)

1.0

In [34]:
# Recall = TP/(TP+FN)
# Recall gives the percentage of positives well predicted by our model.
recall_score(y_test, y_pred_SVC)

0.8015267175572519

In [35]:
# f1_score= (Precition * Recall )/ (Precition + Recall)
f1_score(y_test, y_pred_SVC)

0.8898305084745763

### Logistic Regression Model



In [36]:
# Learn the Logistic Regression Model on train data
from sklearn.linear_model import LogisticRegression
my_model_LR=LogisticRegression()
# Fitting the model in train data set ie the LogisticRegression should learn from the Train Data
my_model_LR.fit(X_train_vector, y_train)

LogisticRegression()

In [37]:
#predicting using LogisticRegression
y_pred_LR = my_model_LR.predict(X_test_vector)
y_pred_LR

array([0, 0, 0, ..., 0, 0, 0])

#### Testing the Logistic Regression Model

In [38]:
# find Accurancy Score = (TP+TN)/(TP+TN+FP+FN)
accuracy_score(y_test, y_pred_LR)

0.97678916827853

In [39]:
# find Precision Score= TP/(TP+FP)
precision_score(y_test, y_pred_LR)

0.9908256880733946

In [40]:
# Recall = TP/(TP+FN)
# Recall gives the percentage of positives well predicted by our model.
recall_score(y_test, y_pred_LR)

0.8244274809160306

In [41]:
# f1_score= (Precition * Recall )/ (Precition + Recall)
f1_score(y_test, y_pred_LR)

0.9000000000000001

---------------THE END-----------------------